In [236]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import datetime as dt
from datetime import timedelta
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score
import statsmodels.api as sm
from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima import auto_arima
std = StandardScaler()

In [237]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [238]:
covid = pd.read_csv("covid_19_data.csv")
covid.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.00000,0.00000,0.00000
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.00000,0.00000,0.00000
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.00000,0.00000,0.00000
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.00000,0.00000,0.00000
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.00000,0.00000,0.00000


In [239]:
#A brief overview of the dataset

print("Size/Shape of the dataset: ", covid.shape)
print("Checking for null values:\n", covid.isnull().sum())
print("Checking Data-type of each column:\n", covid.dtypes)

Size/Shape of the dataset:  (116805, 8)
Checking for null values:
 SNo                    0
ObservationDate        0
Province/State     35353
Country/Region         0
Last Update            0
Confirmed              0
Deaths                 0
Recovered              0
dtype: int64
Checking Data-type of each column:
 SNo                  int64
ObservationDate     object
Province/State      object
Country/Region      object
Last Update         object
Confirmed          float64
Deaths             float64
Recovered          float64
dtype: object


In [240]:
#Dropping column Sno as it is of no use and "Province/State" as it contains contains too many missing values

covid.drop(["SNo","Province/State"], 1, inplace = True)
covid.head()

,ObservationDate,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,01/22/2020,Mainland China,1/22/2020 17:00,1.00000,0.00000,0.00000
1,01/22/2020,Mainland China,1/22/2020 17:00,14.00000,0.00000,0.00000
2,01/22/2020,Mainland China,1/22/2020 17:00,6.00000,0.00000,0.00000
3,01/22/2020,Mainland China,1/22/2020 17:00,1.00000,0.00000,0.00000
4,01/22/2020,Mainland China,1/22/2020 17:00,0.00000,0.00000,0.00000


In [241]:
#Converting "Observation Date" into pandas Datetime format

covid["ObservationDate"] = pd.to_datetime(covid["ObservationDate"])

In [243]:
#Grouping the data by order of Country/Region and then Observation data

country_wise = covid.groupby(["Country/Region","ObservationDate"]).agg({"Confirmed":'sum', "Recovered":'sum', "Deaths":'sum'})

In [244]:
country_wise

Confirmed  Recovered  Deaths
Country/Region                 ObservationDate                              
 Azerbaijan                    2020-02-28         1.00000    0.00000 0.00000
('St. Martin',)                2020-03-10         2.00000    0.00000 0.00000
Afghanistan                    2020-02-24         1.00000    0.00000 0.00000
                               2020-02-25         1.00000    0.00000 0.00000
                               2020-02-26         1.00000    0.00000 0.00000
...                                                   ...        ...     ...
occupied Palestinian territory 2020-03-12         0.00000    0.00000 0.00000
                               2020-03-14         0.00000    0.00000 0.00000
                               2020-03-15         0.00000    0.00000 0.00000
                               2020-03-16         0.00000    0.00000 0.00000
                               2020-03-17         0.00000    0.00000 0.00000

[38831 rows x 3 columns]

In [245]:
#Calculating the active cases 

country_wise["Active Cases"] = country_wise["Confirmed"] - country_wise["Recovered"] - country_wise["Deaths"]
country_wise["log_confirmed"] = np.log(country_wise["Confirmed"])
country_wise["log_active"] = np.log(country_wise["Active Cases"])

# Datewise analysis

In [246]:
#Grouping different types of cases as per the date

date_wise = covid.groupby(["ObservationDate"]).agg({"Confirmed":'sum', "Recovered":'sum', "Deaths":'sum'})
date_wise["Days Since"] = date_wise.index - date_wise.index.min()

In [247]:
date_wise.head()

,Confirmed,Recovered,Deaths,Days Since
ObservationDate,,,,
2020-01-22,555.00000,28.00000,17.00000,0 days
2020-01-23,653.00000,30.00000,18.00000,1 days
2020-01-24,941.00000,36.00000,26.00000,2 days
2020-01-25,1438.00000,39.00000,42.00000,3 days
2020-01-26,2118.00000,52.00000,56.00000,4 days


# Brief overview on the Covid statistics

In [249]:
print("Basic Information")
print("Totol number of countries with Disease Spread: ", len(covid["Country/Region"].unique()))
print("Total number of Confirmed Cases around the World: ", date_wise["Confirmed"].iloc[-1])
print("Total number of Recovered Cases around the World: ", date_wise["Recovered"].iloc[-1])
print("Total number of Deaths Cases around the World: ", date_wise["Deaths"].iloc[-1])
print("Total number of Active Cases around the World: ", (date_wise["Confirmed"].iloc[-1] - date_wise["Recovered"].iloc[-1] - date_wise["Deaths"].iloc[-1]))
print("Total number of Closed Cases around the World: ", date_wise["Recovered"].iloc[-1] + date_wise["Deaths"].iloc[-1])
print("Approximate number of Confirmed Cases per Day around the World: ", np.round(date_wise["Confirmed"].iloc[-1]/ date_wise.shape[0]))
print("Approximate number of Recovered Cases per Day around the World: ", np.round(date_wise["Recovered"].iloc[-1]/ date_wise.shape[0]))
print("Approximate number of Death Cases per Day around the World: ", np.round(date_wise["Deaths"].iloc[-1]/ date_wise.shape[0]))
print("Approximate number of Confirmed Cases per hour around the World: ", np.round(date_wise["Confirmed"].iloc[-1]/ ((date_wise.shape[0])*24)))
print("Approximate number of Recovered Cases per hour around the World: ", np.round(date_wise["Recovered"].iloc[-1]/ ((date_wise.shape[0])*24)))
print("Approximate number of Death Cases per hour around the World: ", np.round(date_wise["Deaths"].iloc[-1]/ ((date_wise.shape[0])*24)))
print("Number of Confirmed Cases in last 24 hours: ", date_wise["Confirmed"].iloc[-1] - date_wise["Confirmed"].iloc[-2])
print("Number of Recovered Cases in last 24 hours: ", date_wise["Recovered"].iloc[-1] - date_wise["Recovered"].iloc[-2])
print("Number of Death Cases in last 24 hours: ", date_wise["Deaths"].iloc[-1] - date_wise["Deaths"].iloc[-2])

Basic Information
Totol number of countries with Disease Spread:  223
Total number of Confirmed Cases around the World:  31779835.0
Total number of Recovered Cases around the World:  21890442.0
Total number of Deaths Cases around the World:  975104.0
Total number of Active Cases around the World:  8914289.0
Total number of Closed Cases around the World:  22865546.0
Approximate number of Confirmed Cases per Day around the World:  129186.0
Approximate number of Recovered Cases per Day around the World:  88986.0
Approximate number of Death Cases per Day around the World:  3964.0
Approximate number of Confirmed Cases per hour around the World:  5383.0
Approximate number of Recovered Cases per hour around the World:  3708.0
Approximate number of Death Cases per hour around the World:  165.0
Number of Confirmed Cases in last 24 hours:  262748.0
Number of Recovered Cases in last 24 hours:  266008.0
Number of Death Cases in last 24 hours:  5526.0


# Time Series Forecasting :

# Holt's Linear Model for confirmed cases

In [311]:
#95% for test data and 5% for training data

model_train = date_wise.iloc[:int(date_wise.shape[0] * 0.95)] 
model_test = date_wise.iloc[int(date_wise.shape[0] * 0.95):]
y_pred = model_test.copy()
train_data = model_train.copy()

In [312]:
#Obtaining the holt linear predictions object using the training data

holt = Holt(np.asarray(model_train["Confirmed"])).fit(smoothing_level = 0.4, smoothing_slope = 0.4, optimized = False)     

In [313]:
#Obtaining the predictions of the holt linear model using test data

model_scores = []
y_pred["Holt"] = holt.forecast(len(model_test))
model_scores.append(np.sqrt(mean_squared_error(y_pred["Confirmed"], y_pred["Holt"])))
print("Root Mean Square Error Holt's Linear Model: ", np.sqrt(mean_squared_error(y_pred["Confirmed"], y_pred["Holt"])))

Root Mean Square Error Holt's Linear Model:  172021.58712565133


In [314]:
#Model's performance on test data

fig = go.Figure()
fig.add_trace(go.Scatter(x = model_test.index, y = model_test["Confirmed"],
                    mode = 'lines + markers', name = "Test Data for Confirmed Cases",))
fig.add_trace(go.Scatter(x = model_test.index, y = y_pred["Holt"],
                    mode = 'lines + markers', name = "Prediction of Confirmed Cases",))
fig.update_layout(title = "Confirmed Cases Holt's Linear Model Prediction",
                 xaxis_title = "Date", yaxis_title = "Confirmed Cases", legend = dict(x = 0, y = 1, traceorder = "normal"))
fig.show()

In [315]:
#Creating a dataframe to compare the predictions of all the models


holt_new_prediction = []
for i in range(1,18):
    holt_new_prediction.append(holt.forecast((len(model_test) + i))[-1])

model_predictions = pd.DataFrame(zip(holt_new_date),
                               columns = ["Dates"])

In [316]:
model_predictions["Holt's Linear Model Prediction"] = holt_new_prediction
model_predictions.head()

,Dates,Holt's Linear Model Prediction
0,2020-09-24,31773237.70333
1,2020-09-25,32033255.13782
2,2020-09-26,32293272.57231
3,2020-09-27,32553290.00680
4,2020-09-28,32813307.44129


# Holt's Winter Model for Daily Time Series

In [317]:
#Performing exponential smoothing

es = ExponentialSmoothing(np.asarray(model_train['Confirmed']),seasonal_periods = 14, trend = 'add', seasonal = 'mul').fit()

In [318]:
#Root mean squared error for Holt's Winter Model

y_pred["Holt's Winter Model"] = es.forecast(len(model_test))
model_scores.append(np.sqrt(mean_squared_error(y_pred["Confirmed"], y_pred["Holt's Winter Model"])))
print("Root Mean Square Error for Holt's Winter Model: ", np.sqrt(mean_squared_error(y_pred["Confirmed"], y_pred["Holt's Winter Model"])))

Root Mean Square Error for Holt's Winter Model:  133346.20617539258


In [319]:
#Obtaining the predictions of the holt winter model using test data

fig = go.Figure()
fig.add_trace(go.Scatter(x = model_test.index, y = model_test["Confirmed"],
                    mode = 'lines + markers', name = "Test Data for Confirmed Cases",))
fig.add_trace(go.Scatter(x = model_test.index, y = y_pred["Holt\'s Winter Model"],
                    mode = 'lines + markers', name = "Prediction of Confirmed Cases",))
fig.update_layout(title = "Confirmed Cases Holt's Winter Model Prediction",
                 xaxis_title = "Date", yaxis_title = "Confirmed Cases", legend = dict(x = 0, y = 1, traceorder = "normal"))
fig.show()

In [320]:
holt_winter_new_prediction = []
for i in range(1,18):
    holt_winter_new_prediction.append(es.forecast((len(model_test) + i))[-1])
model_predictions["Holt's Winter Model Prediction"] = holt_winter_new_prediction
model_predictions.head()

,Dates,Holt's Linear Model Prediction,Holt's Winter Model Prediction
0,2020-09-24,31773237.70333,31828540.48574
1,2020-09-25,32033255.13782,32138941.49085
2,2020-09-26,32293272.57231,32416136.38668
3,2020-09-27,32553290.00680,32641361.50132
4,2020-09-28,32813307.44129,32896121.38189


# AR Model (using AUTO ARIMA)

In [321]:
#Training the AR model

model_ar = auto_arima(model_train["Confirmed"], trace = True, error_action = 'ignore', start_p = 0, start_q = 0, max_p = 4, max_q = 0,
                   suppress_warnings = True,stepwise = False, seasonal = False)
model_ar.fit(model_train["Confirmed"])

 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=5238.720, Time=0.02 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=5226.902, Time=0.04 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=5227.881, Time=0.03 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=5188.886, Time=0.07 sec
 ARIMA(4,2,0)(0,0,0)[0] intercept   : AIC=5143.415, Time=0.08 sec
Total fit time: 0.238 seconds


ARIMA(maxiter=50, method='lbfgs', order=(4, 2, 0), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True)

In [322]:
#Obtaining predictions using AR

prediction_ar = model_ar.predict(len(model_test))
y_pred["AR Model Prediction"] = prediction_ar

In [323]:
#RMSE for AR model

model_scores.append(np.sqrt(mean_squared_error(y_pred["Confirmed"], y_pred["AR Model Prediction"])))
print("Root Mean Square Error for AR Model: ", np.sqrt(mean_squared_error(y_pred["Confirmed"], y_pred["AR Model Prediction"])))

Root Mean Square Error for AR Model:  37250.75213354712


In [324]:
#Making the AR model predictions using test data

fig = go.Figure()
fig.add_trace(go.Scatter(x = model_test.index, y = model_test["Confirmed"],
                    mode = 'lines + markers', name = "Test data for Confirmed Cases",))
fig.add_trace(go.Scatter(x = model_test.index, y = y_pred["AR Model Prediction"],
                    mode = 'lines + markers', name = "Prediction of Confirmed Cases",))
fig.update_layout(title = "Confirmed Cases AR Model Prediction",
                 xaxis_title = "Date", yaxis_title = "Confirmed Cases", legend = dict(x = 0, y = 1, traceorder = "normal"))
fig.show()

In [325]:
AR_model_new_prediction = []
for i in range(1,18):
    AR_model_new_prediction.append(model_ar.predict(len(model_test) + i)[-1])
model_predictions["AR Model Prediction"] = AR_model_new_prediction
model_predictions.head()

,Dates,Holt's Linear Model Prediction,Holt's Winter Model Prediction,AR Model Prediction
0,2020-09-24,31773237.70333,31828540.48574,32075553.86726
1,2020-09-25,32033255.13782,32138941.49085,32361187.55458
2,2020-09-26,32293272.57231,32416136.38668,32646427.72161
3,2020-09-27,32553290.00680,32641361.50132,32935237.72275
4,2020-09-28,32813307.44129,32896121.38189,33228816.04745


# MA Model (using AUTO ARIMA)

In [326]:
#Training the MA model

model_ma = auto_arima(model_train["Confirmed"], trace = True, error_action = 'ignore', start_p = 0, start_q = 0, max_p = 0, max_q = 2,
                   suppress_warnings = True, stepwise = False, seasonal = False)
model_ma.fit(model_train["Confirmed"])

 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=5238.720, Time=0.02 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=5218.406, Time=0.05 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AIC=5226.234, Time=0.07 sec
Total fit time: 0.132 seconds


ARIMA(maxiter=50, method='lbfgs', order=(0, 2, 1), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True)

In [327]:
#Obtaining predictions using MA model

prediction_ma = model_ma.predict(len(model_test))
y_pred["MA Model Prediction"] = prediction_ma

In [328]:
#RMSE for MA model

model_scores.append(np.sqrt(mean_squared_error(model_test["Confirmed"], prediction_ma)))
print("Root Mean Square Error for MA Model: ", np.sqrt(mean_squared_error(model_test["Confirmed"], prediction_ma)))

Root Mean Square Error for MA Model:  62944.74735014162


In [329]:
#Checking performance of MA model on the test data

fig = go.Figure()
fig.add_trace(go.Scatter(x = model_test.index, y = model_test["Confirmed"],
                    mode = 'lines + markers', name = "Test data for Confirmed Cases",))
fig.add_trace(go.Scatter(x = model_test.index, y = y_pred["MA Model Prediction"],
                    mode = 'lines + markers', name = "Prediction for Confirmed Cases",))
fig.update_layout(title = "Confirmed Cases MA Model Prediction",
                 xaxis_title = "Date", yaxis_title = "Confirmed Cases", legend = dict(x = 0, y = 1, traceorder = "normal"))
fig.show()

In [330]:
MA_model_new_prediction = []
for i in range(1,18):
    MA_model_new_prediction.append(model_ma.predict(len(model_test) + i)[-1])
model_predictions["MA Model Prediction"] = MA_model_new_prediction
model_predictions.head()

,Dates,Holt's Linear Model Prediction,Holt's Winter Model Prediction,AR Model Prediction,MA Model Prediction
0,2020-09-24,31773237.70333,31828540.48574,32075553.86726,32179130.35049
1,2020-09-25,32033255.13782,32138941.49085,32361187.55458,32477986.83185
2,2020-09-26,32293272.57231,32416136.38668,32646427.72161,32778431.36452
3,2020-09-27,32553290.00680,32641361.50132,32935237.72275,33080463.94851
4,2020-09-28,32813307.44129,32896121.38189,33228816.04745,33384084.58383


# ARIMA Model (using AUTOARIMA)

In [332]:
#Training the ARIMA model

model_arima = auto_arima(model_train["Confirmed"], trace = True, error_action='ignore', start_p = 1, start_q = 1, max_p = 3, max_q = 3,
                   suppress_warnings = True, stepwise = False, seasonal = False)
model_arima.fit(model_train["Confirmed"])

 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=5238.720, Time=0.02 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=5218.406, Time=0.05 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AIC=5226.234, Time=0.05 sec
 ARIMA(0,2,3)(0,0,0)[0] intercept   : AIC=5226.683, Time=0.10 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=5226.902, Time=0.03 sec
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=5220.548, Time=0.09 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=5375.063, Time=0.21 sec
 ARIMA(1,2,3)(0,0,0)[0] intercept   : AIC=5212.702, Time=0.11 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=5227.881, Time=0.03 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=5222.985, Time=0.23 sec
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=5242.486, Time=0.27 sec
 ARIMA(2,2,3)(0,0,0)[0] intercept   : AIC=5208.092, Time=0.18 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=5188.886, Time=0.07 sec
 ARIMA(3,2,1)(0,0,0)[0] intercept   : AIC=5133.067, Time=0.15 sec
 ARIMA(3,2,2)(0,0,0)[0] intercept   : AIC=5086.450, Time=0.48 sec
Total fit 

ARIMA(maxiter=50, method='lbfgs', order=(3, 2, 2), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True)

In [333]:
#Obtaining the predictions for the ARIMA model

prediction_arima = model_arima.predict(len(model_test))
y_pred["ARIMA Model Prediction"] = prediction_arima

In [334]:
#RMSE for ARIMA 

model_scores.append(np.sqrt(mean_squared_error(model_test["Confirmed"], prediction_arima)))
print("Root Mean Square Error for ARIMA Model: ", np.sqrt(mean_squared_error(model_test["Confirmed"], prediction_arima)))

Root Mean Square Error for ARIMA Model:  116832.60073380142


In [335]:
#Checking performance of ARIMA on the test data

fig = go.Figure()
fig.add_trace(go.Scatter(x = model_test.index, y = model_test["Confirmed"],
                    mode = 'lines + markers', name = "Test data for Confirmed Cases",))
fig.add_trace(go.Scatter(x = model_test.index, y = y_pred["ARIMA Model Prediction"],
                    mode = 'lines + markers', name = "Prediction for Confirmed Cases",))
fig.update_layout(title = "Confirmed Cases ARIMA Model Prediction",
                 xaxis_title = "Date", yaxis_title = "Confirmed Cases",legend = dict(x = 0, y = 1, traceorder = "normal"))
fig.show()

In [336]:
ARIMA_model_new_prediction = []
for i in range(1,18):
    ARIMA_model_new_prediction.append(model_arima.predict(len(model_test) + i)[-1])
model_predictions["ARIMA Model Prediction"] = ARIMA_model_new_prediction
model_predictions.head()

,Dates,Holt's Linear Model Prediction,Holt's Winter Model Prediction,AR Model Prediction,MA Model Prediction,ARIMA Model Prediction
0,2020-09-24,31773237.70333,31828540.48574,32075553.86726,32179130.35049,32377605.37248
1,2020-09-25,32033255.13782,32138941.49085,32361187.55458,32477986.83185,32730517.86499
2,2020-09-26,32293272.57231,32416136.38668,32646427.72161,32778431.36452,33061626.97862
3,2020-09-27,32553290.00680,32641361.50132,32935237.72275,33080463.94851,33373328.13263
4,2020-09-28,32813307.44129,32896121.38189,33228816.04745,33384084.58383,33685437.87970


# SARIMA Model (using AUTO ARIMA)

In [337]:
#Training the SARIMA model

model_sarima = auto_arima(model_train["Confirmed"], trace = True, error_action = 'ignore', 
                         start_p = 0, start_q = 0, max_p = 2, max_q = 2, m = 7,
                   suppress_warnings = True, stepwise = True, seasonal = True)
model_sarima.fit(model_train["Confirmed"])

Performing stepwise search to minimize aic
 ARIMA(0,2,0)(1,0,1)[7]             : AIC=5171.866, Time=0.30 sec
 ARIMA(0,2,0)(0,0,0)[7]             : AIC=5237.673, Time=0.00 sec
 ARIMA(1,2,0)(1,0,0)[7]             : AIC=5137.797, Time=0.10 sec
 ARIMA(0,2,1)(0,0,1)[7]             : AIC=5179.694, Time=0.06 sec
 ARIMA(1,2,0)(0,0,0)[7]             : AIC=5226.426, Time=0.01 sec
 ARIMA(1,2,0)(2,0,0)[7]             : AIC=5123.124, Time=0.16 sec
 ARIMA(1,2,0)(2,0,1)[7]             : AIC=5117.732, Time=0.31 sec
 ARIMA(1,2,0)(1,0,1)[7]             : AIC=5063.135, Time=0.44 sec
 ARIMA(1,2,0)(0,0,1)[7]             : AIC=5189.197, Time=0.08 sec
 ARIMA(1,2,0)(1,0,2)[7]             : AIC=5117.727, Time=0.27 sec
 ARIMA(1,2,0)(0,0,2)[7]             : AIC=5160.948, Time=0.13 sec
 ARIMA(1,2,0)(2,0,2)[7]             : AIC=5119.085, Time=0.33 sec
 ARIMA(2,2,0)(1,0,1)[7]             : AIC=5060.706, Time=0.68 sec
 ARIMA(2,2,0)(0,0,1)[7]             : AIC=5189.370, Time=0.07 sec
 ARIMA(2,2,0)(1,0,0)[7]          

ARIMA(maxiter=50, method='lbfgs', order=(2, 2, 0), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(1, 0, 1, 7),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=False)

In [338]:
#Obtaining the predictions for SARIMA model

prediction_sarima = model_sarima.predict(len(model_test))
y_pred["SARIMA Model Prediction"] = prediction_sarima

In [339]:
#RMSE for SARIMA model

model_scores.append(np.sqrt(mean_squared_error(y_pred["Confirmed"], y_pred["SARIMA Model Prediction"])))
print("Root Mean Square Error for SARIMA Model: ", np.sqrt(mean_squared_error(y_pred["Confirmed"], y_pred["SARIMA Model Prediction"])))

Root Mean Square Error for SARIMA Model:  37117.83555893187


In [340]:
#Checking the performance of SARIMA on the test data

fig = go.Figure()
fig.add_trace(go.Scatter(x = model_test.index, y = model_test["Confirmed"],
                    mode = 'lines', name = "Test data for Confirmed Cases",))
fig.add_trace(go.Scatter(x = model_test.index, y = y_pred["SARIMA Model Prediction"],
                    mode = 'lines', name = "Prediction for Confirmed Cases",))
fig.update_layout(title = "Confirmed Cases SARIMA Model Prediction",
                 xaxis_title = "Date", yaxis_title = "Confirmed Cases", legend = dict(x = 0, y = 1, traceorder = "normal"))
fig.show()

In [341]:
SARIMA_model_new_prediction = []
for i in range(1,18):
    SARIMA_model_new_prediction.append(model_sarima.predict(len(model_test) + i)[-1])
model_predictions["SARIMA Model Prediction"] = SARIMA_model_new_prediction
model_predictions.head()

,Dates,Holt's Linear Model Prediction,Holt's Winter Model Prediction,AR Model Prediction,MA Model Prediction,ARIMA Model Prediction,SARIMA Model Prediction
0,2020-09-24,31773237.70333,31828540.48574,32075553.86726,32179130.35049,32377605.37248,32094470.40457
1,2020-09-25,32033255.13782,32138941.49085,32361187.55458,32477986.83185,32730517.86499,32409710.66874
2,2020-09-26,32293272.57231,32416136.38668,32646427.72161,32778431.36452,33061626.97862,32700383.04969
3,2020-09-27,32553290.00680,32641361.50132,32935237.72275,33080463.94851,33373328.13263,32958960.42472
4,2020-09-28,32813307.44129,32896121.38189,33228816.04745,33384084.58383,33685437.87970,33225925.57036


# Performance of each model by root mean squared error

In [342]:
model_names = ["Holt's Linear","Holt's Winter Model",
            "Auto Regressive Model (AR)","Moving Average Model (MA)","ARIMA Model","SARIMA Model"]
model_summary = pd.DataFrame(zip(model_names, model_scores), columns = ["Model Name","Root Mean Squared Error"]).sort_values(["Root Mean Squared Error"])
model_summary

,Model Name,Root Mean Squared Error
5,SARIMA Model,37117.83556
2,Auto Regressive Model (AR),37250.75213
3,Moving Average Model (MA),62944.74735
4,ARIMA Model,116832.60073
1,Holt's Winter Model,133346.20618
0,Holt's Linear,172021.58713


#### SARIMA is the best model for this dataset as it gives us the lowest RMSE value.

# Time Series Forecasting for Death Cases

In [343]:
#Visualisation of deaths (Training data)

fig = go.Figure()
fig.add_trace(go.Scatter(x = model_train.index, y = model_train["Deaths"],
                    mode = 'lines', name = "Death Cases"))
fig.update_layout(title = "Death Cases",
                 xaxis_title = "Date", yaxis_title = "Number of Death Cases", legend = dict(x = 0, y = 1, traceorder = "normal"))
fig.show()

In [ ]:
#Training the ARIMA model to predict number of deaths

model_arima_deaths = auto_arima(model_train["Deaths"], trace = True, error_action = 'ignore', start_p = 0, start_q = 0,
                              max_p = 5, max_q = 5, suppress_warnings = True, stepwise = False, seasonal = False)     
model_arima_deaths.fit(model_train["Deaths"])

 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=4002.845, Time=0.02 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=3964.048, Time=0.17 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AIC=3948.317, Time=0.15 sec
 ARIMA(0,2,3)(0,0,0)[0] intercept   : AIC=3941.841, Time=0.22 sec
 ARIMA(0,2,4)(0,0,0)[0] intercept   : AIC=3911.075, Time=0.39 sec
 ARIMA(0,2,5)(0,0,0)[0] intercept   : AIC=3909.079, Time=0.52 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=3989.453, Time=0.03 sec
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=3947.299, Time=0.25 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=3948.415, Time=0.18 sec
 ARIMA(1,2,3)(0,0,0)[0] intercept   : AIC=3937.029, Time=0.35 sec
 ARIMA(1,2,4)(0,0,0)[0] intercept   : AIC=3904.084, Time=0.60 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=3990.160, Time=0.05 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=3946.507, Time=0.26 sec
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=3946.727, Time=0.50 sec
 ARIMA(2,2,3)(0,0,0)[0] intercept   : AIC=3877.601, Time=0.45 sec
 ARIMA(3,2

In [296]:
#Forming the predictions for ARIMA

predictions_deaths_arima = model_arima_deaths.predict(len(model_test))
y_pred["ARIMA Death Prediction"] = predictions_deaths_arima

In [297]:
#RMSE for ARIMA

model_death_scores = []
model_death_scores.append(np.sqrt(mean_squared_error(y_pred["Deaths"], y_pred["ARIMA Death Prediction"])))
print("Root Mean Square Error for deaths (ARIMA): ", np.sqrt(mean_squared_error(model_test["Deaths"], predictions_deaths_arima)))

Root Mean Square Error for deaths (ARIMA):  10221.868253267872


In [298]:
#Checking the performance on test dataset

fig = go.Figure()
fig.add_trace(go.Scatter(x = model_test.index, y = model_test["Deaths"],
                    mode = 'lines + markers', name = "Test data for Death Cases",))
fig.add_trace(go.Scatter(x = model_test.index, y = y_pred["ARIMA Death Prediction"],
                    mode = 'lines + markers', name = "Prediction for Death Cases",))
fig.update_layout(title = "Death Cases ARIMA Model Prediction",
                 xaxis_title = "Date", yaxis_title = "Death Cases", legend = dict(x = 0, y = 1, traceorder = "normal"))
fig.show()

In [299]:
#Training the SARIMA model

model_sarima_deaths = auto_arima(model_train["Deaths"], trace = True, error_action = 'ignore', 
                         start_p = 0, start_q = 0, max_p = 2, max_q = 2, m = 7,
                   suppress_warnings = True, stepwise = True, seasonal = True)
model_sarima_deaths.fit(model_train["Deaths"])

Performing stepwise search to minimize aic
 ARIMA(0,2,0)(1,0,1)[7]             : AIC=3964.604, Time=0.20 sec
 ARIMA(0,2,0)(0,0,0)[7]             : AIC=4000.929, Time=0.00 sec
 ARIMA(1,2,0)(1,0,0)[7]             : AIC=3937.841, Time=0.12 sec
 ARIMA(0,2,1)(0,0,1)[7]             : AIC=3926.731, Time=0.14 sec
 ARIMA(0,2,1)(0,0,0)[7]             : AIC=3963.507, Time=0.07 sec
 ARIMA(0,2,1)(1,0,1)[7]             : AIC=3876.148, Time=0.36 sec
 ARIMA(0,2,1)(1,0,0)[7]             : AIC=3892.531, Time=0.19 sec
 ARIMA(0,2,1)(2,0,1)[7]             : AIC=3869.364, Time=0.57 sec
 ARIMA(0,2,1)(2,0,0)[7]             : AIC=3873.416, Time=0.27 sec
 ARIMA(0,2,1)(2,0,2)[7]             : AIC=3871.124, Time=0.92 sec
 ARIMA(0,2,1)(1,0,2)[7]             : AIC=3877.648, Time=0.94 sec
 ARIMA(0,2,0)(2,0,1)[7]             : AIC=3958.895, Time=0.36 sec
 ARIMA(1,2,1)(2,0,1)[7]             : AIC=3870.446, Time=0.82 sec
 ARIMA(0,2,2)(2,0,1)[7]             : AIC=3870.543, Time=0.57 sec
 ARIMA(1,2,0)(2,0,1)[7]          

ARIMA(maxiter=50, method='lbfgs', order=(0, 2, 1), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(2, 0, 1, 7),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=False)

In [300]:
#Forming the predictions using SARIMA model obtained

predictions_deaths_sarima = model_sarima_deaths.predict(len(model_test))
y_pred["SARIMA Death Prediction"] = predictions_deaths_sarima

In [301]:
#RMSE for SARIMA

print("Root Mean Square Error: ", np.sqrt(mean_squared_error(model_test["Deaths"], predictions_deaths_sarima)))

Root Mean Square Error:  7269.6054501862645


In [302]:
#Checking the performance on test data

fig = go.Figure()
fig.add_trace(go.Scatter(x = model_test.index, y = model_test["Deaths"],
                    mode='lines + markers', name = "Test data for Death Cases",))
fig.add_trace(go.Scatter(x = model_test.index, y = y_pred["SARIMA Death Prediction"],
                    mode = 'lines + markers', name = "Prediction for Death Cases",))
fig.update_layout(title = "Death Cases SARIMA Model Prediction",
                 xaxis_title = "Date", yaxis_title = "Death Cases", legend = dict(x = 0, y = 1, traceorder = "normal"))
fig.show()

In [303]:
death_forecast_arima = []
death_forecast_sarima = []
for i in range(1,18):
    death_forecast_arima.append(model_arima_deaths.predict(len(model_test) + i)[-1])
for i in range(1,18):
    death_forecast_sarima.append(model_sarima_deaths.predict(len(model_test) + i)[-1])


model_death_predictions = pd.DataFrame(zip(holt_new_date), columns = ["Dates"])
model_death_predictions["ARIMA Model Prediction"] = death_forecast_arima
model_death_predictions["SARIMA Model Prediction"] = death_forecast_sarima

In [304]:
model_death_predictions.head()

,Dates,ARIMA Model Prediction,SARIMA Model Prediction
0,2020-09-24,998686.03497,993530.11066
1,2020-09-25,1005210.71666,999885.58007
2,2020-09-26,1011853.11354,1005891.13664
3,2020-09-27,1018639.46525,1011513.69244
4,2020-09-28,1025515.31946,1017760.11190


# Performance of the models by RMSE

In [305]:
model_death_scores = []
model_death_scores.append(np.sqrt(mean_squared_error(model_test["Deaths"], predictions_deaths_arima)))
model_death_scores.append(np.sqrt(mean_squared_error(model_test["Deaths"], predictions_deaths_sarima)))
model_names = ["ARIMA model death predictions","SARIMA model death predictions"]

In [306]:
death_summary = pd.DataFrame(zip(model_names, model_death_scores), columns = ["Model Name","Root Mean Squared Error"]).sort_values(["Root Mean Squared Error"])
death_summary

,Model Name,Root Mean Squared Error
1,SARIMA model death predictions,7269.60545
0,ARIMA model death predictions,10221.86825


#### SARIMA is a better model to predict deaths